<a href="https://colab.research.google.com/github/s-ribansky/RedditProject_NLPDL/blob/master/Code_final_report_Reddit_%26_Twitter_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
! git clone https://github.com/NVIDIA/apex
os.chdir('apex')
!pip install -v --no-cache-dir ./

Cloning into 'apex'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6593 (delta 0), reused 2 (delta 0), pack-reused 6589
Receiving objects: 100% (6593/6593), 13.70 MiB | 13.77 MiB/s, done.
Resolving deltas: 100% (4383/4383), done.
Created temporary directory: /tmp/pip-ephem-wheel-cache-ciyee2kq
Created temporary directory: /tmp/pip-req-tracker-9zf6xh5d
Created requirements tracker '/tmp/pip-req-tracker-9zf6xh5d'
Created temporary directory: /tmp/pip-install-62vwgut7
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-fqo8ipzb
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-9zf6xh5d'
    Running setup.py (path:/tmp/pip-req-build-fqo8ipzb/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.4.0
    running egg_info
    creating /tmp/pip-req-build-fqo8ipzb/pip-egg-info/apex.

In [0]:
# Uncomment for extra ram access sometimes
#a=[]
#while(1):
  #a.append('1')

In [0]:
#Do in console : To prevent colab timeouts (smaller than 12 hours), we can run the following in the page console. To access this, right click on the page and choose Inspect (Command + Shift + C for Mac users). Then click on Console and paste the following code snippet:

#function ClickConnect(){
#console.log("Working");
#document.querySelector("colab-toolbar-button").click()
#}setInterval(ClickConnect,50000)

In [4]:
!pip install simpletransformers
!pip install transformers


     |████████████████████████████████| 163kB 6.9MB/s 
     |████████████████████████████████| 3.8MB 17.7MB/s 
     |████████████████████████████████| 573kB 47.2MB/s 
     |████████████████████████████████| 204kB 50.4MB/s 
     |████████████████████████████████| 1.0MB 46.2MB/s 
     |████████████████████████████████| 890kB 42.5MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=ae01913c2d6864712587c71352a6edda84984bcb023ee09d2ef23cbe614686f4
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=e51f5e186040acd9f8340c9bf82e1ea0c645aab9c4a30fd95bf5d25bca4e2694
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built seqeval sacremoses
ERROR: transformers 2.8.0 has requirement tokenizers==0.5.2, but you'll have tokenizers 0.7.0 which

In [0]:
from argparse import Namespace
from collections import Counter
import json
import os
import string
import sklearn
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
from apex import amp
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import random


In [0]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Load Twitterdata:
dataedit contains all tweets aggregated for controversiality;
dataedit_balanced (generated later) contains a random sample of tweets with the same ratio of controversiality as the (sampled) reddit dataset

In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/database2.csv') # Redditdata

In [0]:
twitterdataraw=pd.read_pickle('/content/drive/My Drive/Colab Notebooks/labeled_data.p') # Raw Twitterdata

In [0]:
dataedit=pd.DataFrame(twitterdataraw.iloc[:,5]) # Load all text into dataedit
dataedit.columns=['text']
dataedit.insert(1,"labels",np.ones(len(twitterdataraw),dtype=int)) # load labels initially all ones
indx=np.where(twitterdataraw.iloc[:,4]==2) # find index where tweet is not controversial
dataedit.iloc[indx[0],1]=0 #assign 0 to the places with the uncontroversial index


In [15]:
#Test to validate if data looks good
print("Twitter data, aggregated for controversiality")
print(dataedit)

Twitter data, aggregated for controversiality
                                                    text  labels
0      !!! RT @mayasolovely: As a woman you shouldn't...       0
1      !!!!! RT @mleew17: boy dats cold...tyga dwn ba...       1
2      !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...       1
3      !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...       1
4      !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...       1
...                                                  ...     ...
25291  you's a muthaf***in lie &#8220;@LifeAsKing: @2...       1
25292  you've gone and broke the wrong heart baby, an...       0
25294  young buck wanna eat!!.. dat nigguh like I ain...       1
25295              youu got wild bitches tellin you lies       1
25296  ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...       0

[24783 rows x 2 columns]


In [16]:
#Validate Datatypes
dataedit['text'] = dataedit['text'].astype(str)
print (dataedit['text'].dtype)
print (dataedit['labels'].dtype)

object
int64


In [20]:
print("df (redditdata) Shape: ",df.shape)
print("twitterdata nr of controversial tweets: ",sum(dataedit.iloc[:,1]))
print("twitterdata Ratio of controversial tweets",sum(dataedit.iloc[:,1])/dataedit.shape[0])
print("Redditdata ratio of controversial tweets",sum(df.iloc[:,1])/df.shape[0])
ids=np.where(twitterdataraw.iloc[:,4]==2)[0] # find all non controversial tweets
#print(ids.shape)
i=0
while(i<2780): # reach similar ratio
  randomtmp=np.array(random.randrange(len(dataedit))) # randomly select one tweet
  if np.any(np.in1d(ids,randomtmp))==0: # check if that tweet already is not yet selected (as are all uncotroversial)
    ids=np.append(ids,randomtmp) # if unique add it 
    i+=1
print("IDS Shape after resampling: ",ids.shape)
dataedit_balanced=dataedit.iloc[ids,:]
#print(dataedit_balanced)
print("Ratio of controversial tweets after resampling",sum(dataedit_balanced.iloc[:,1])/dataedit_balanced.shape[0])

df (redditdata) Shape:  (23404, 2)
twitterdata nr of controversial tweets:  20620
twitterdata Ratio of controversial tweets 0.8320219505306057
Redditdata ratio of controversial tweets 0.39506067338916423
IDS Shape after resampling:  (6943,)
Ratio of controversial tweets after resampling 0.4004032838830477


In [14]:
#validate datatypes
df['text'] = df['text'].astype(str)
print (df['text'].dtype)
print (df['labels'].dtype)


object
int64


In [23]:
# Select a random smaple of the redditdata in reduced_dataset
print("Redditdata original shape",df.shape)
ids=random.sample(range(0, df.shape[0]), 6000)
reduced_dataset=df.iloc[ids,:]
print("Redditdata random sample shape",reduced_dataset.shape)
#print(reduced_dataset)
print("nr of controversial reddit post in random sample (rerun cell for change)",sum(reduced_dataset.iloc[:,1]))

Redditdata original shape (23404, 2)
Redditdata random sample shape (6000, 2)
nr of controversial reddit post in random sample (rerun cell for change) 2361


Split data into training and test set. Select which data to use.

In [25]:
# Redditdata Random downsampled:
#df_train, df_eval = train_test_split(reduced_dataset, test_size=0.2)
#Reddit data full:
#df_train, df_eval = train_test_split(df, test_size=0.2)
#Twitter data, downsampled for same controversiality ratio:
df_train, df_eval = train_test_split(dataedit_balanced, test_size=0.2)
#Full twitterdata:
#df_train, df_eval = train_test_split(dataedit, test_size=0.2)

print (df['text'].dtype)

object


In [27]:
#validate datatypes
train_df = df_train
eval_df = df_eval
print("Shape training data",train_df.shape)
print("Shape evaluation data",eval_df.shape)


Shape training data (5554, 2)
Shape evaluation data (1389, 2)


Load pretrained distilbert or previous model

In [28]:
model = ClassificationModel('distilbert', 'distilbert-base-uncased', use_cuda=True) #pretrained Distilbert
#model = ClassificationModel('distilbert', '/content/drive/My Drive/greatmodel/', use_cuda=True) #pretrained model8

INFO:filelock:Lock 139705375218096 acquired on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock


INFO:filelock:Lock 139705375218096 released on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock


INFO:filelock:Lock 139705405091680 acquired on /root/.cache/torch/transformers/7b8a8f0b21c4e7f6962451c9370a5d9af90372a5f64637a251f2de154d0fc72c.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5.lock


INFO:filelock:Lock 139705405091680 released on /root/.cache/torch/transformers/7b8a8f0b21c4e7f6962451c9370a5d9af90372a5f64637a251f2de154d0fc72c.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5.lock


INFO:filelock:Lock 139705375217536 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


INFO:filelock:Lock 139705375217536 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


Set options for training

In [0]:
arguments={
      "sliding_window": True,
      "reprocess_input_data": True,
      "overwrite_output_dir": True,
      "stride": 0.8,
      "max_seq_length": 128,
      "best_model_dir": "/content/drive/My Drive/bestmodel/",
      "cache_dir": "/content/drive/My Drive/cache/",
      "output_dir": "/content/drive/My Drive/output/",
      "save_steps": train_df.shape[0],

      "logging_steps": eval_df.shape[0],
      "train_batch_size": 8,
      "eval_batch_size": 8,
      "gradient_accumulation_steps": 1,
      "num_train_epochs": 1,
      "weight_decay": 0,
      "learning_rate": 4e-5,
      "evaluate_during_training":True,
      "evaluate_during_training_steps": eval_df.shape[0],
      "evaluate_during_training_verbose": True,
      "use_early_stopping": True,
      "early_stopping_patience": 3,
      "early_stopping_delta": 0,
      "early_stopping_metric": "eval_loss",
      "early_stopping_metric_minimize": True,
      "loss_scale":'dynamic'}
#gradient_accumulation_steps could save disk space for slower training

Train model

In [30]:
model.train_model(train_df,eval_df=eval_df, args=arguments)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


INFO:simpletransformers.classification.classification_model: 5563 features created from 5554 samples.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.711602

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:113: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.141940Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.234450Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.004747


INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled
INFO:simpletransformers.classification.classification_model: 1389 features created from 1389 samples.
INFO:simpletransformers.classification.classification_model:{'mcc': 0.9022585082734723, 'tp': 532, 'tn': 791, 'fp': 20, 'fn': 46, 'eval_loss': 0.20696139392309956}


INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to /content/drive/My Drive/output/.


In [21]:
# Evaluate on entire twitter set
#entire twitter set:
#result, model_outputs, wrong_predictions = model.eval_model(dataedit, acc=sklearn.metrics.accuracy_score)

#balanced twitter set:
#result, model_outputs, wrong_predictions = model.eval_model(dataedit_balanced, acc=sklearn.metrics.accuracy_score)
#Evaluation data
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


INFO:simpletransformers.classification.classification_model: 1389 features created from 1389 samples.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9075762913372439, 'tp': 503, 'tn': 825, 'fp': 21, 'fn': 40, 'eval_loss': 0.1687867278340219, 'acc': 0.9560835133189345}


In [22]:
x_predict = input("Hello, how are you doing?")

Hello, how are you doing?good


In [23]:
predictions, raw_outputs = model.predict([x_predict])
print(predictions)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


INFO:simpletransformers.classification.classification_model: 1 features created from 1 samples.



[0]
